In [1]:
import cadquery as cq
from cadquery import exporters
from jupyter_cadquery.cadquery import (PartGroup, Part, Edges, Faces, Vertices, show, 
                                       replay, enable_replay, disable_replay, reset_replay)
from jupyter_cadquery import set_sidecar, set_defaults

from math import sin, radians, sqrt

import ipywidgets as widgets

Overwriting auto display for cadquery Workplane and Shape


In [2]:
set_defaults(axes=True, grid=True, axes0=True, timeit=False, cad_width=640, height=800)

set_sidecar("Z", init=True)
replay_box = widgets.Checkbox(False, description='Enable Replay')
out = widgets.Output()
display(replay_box)
display(out)

@out.capture()
def changed(e):
    if e["new"]:
        enable_replay(warning=False)
    else:
        disable_replay()

replay_box.observe(changed, names='value')

Checkbox(value=False, description='Enable Replay')

Output()

In [3]:
d1 = 11.8

linear_rod_spacing = 99
linear_rod_d = 12
linear_rod_bearing_d = 21

screw_r = 8.2/2

bearing_d = 22
bearing_h = 7
bearing_y_offset = 1

min_thickness = 4

In [4]:
def bearing_void(h1, h2=None):
    if h2 == None:
        h2 = h1
    return (
        cq.Workplane("XY")
        .cylinder(bearing_h, bearing_d/2)
        .intersect(cq.Workplane("XY").sphere(bearing_d/2))
        .faces(">Z")
        .workplane()
        .cylinder(h1, bearing_d/2-3, centered=(True,True,False))
        .faces("<Z")
        .workplane()
        .cylinder(h2, screw_r+1, centered=(True,True,False))
    )

replay(bearing_void(10))

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [5]:
m5_r = 5/2
m5_hole_depth = 7
m5_head_r = 8.5/2
m5_nut = (10.4, 6)

def m5_hole(h):
    return (
        cq.Workplane("XZ")
        .cylinder(m5_hole_depth, m5_r)
        .faces(">Y")
        .workplane()
        .cylinder(h, m5_head_r, centered=(True, True, False))
        .faces("<Y")
        .workplane()
        .box(m5_nut[0], m5_nut[1], h, centered=(True,True,False))
    )

replay(m5_hole(10))

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [6]:
body_y = bearing_d+bearing_y_offset+min_thickness*2
x_extra = (body_y-linear_rod_d)/2
body_x = x_extra*2+linear_rod_d+linear_rod_spacing
body_z = bearing_h + min_thickness*2

body = (
    cq.Workplane("XY")
    .box(body_x, body_y, body_z)
    .edges()
    .chamfer(1)
    .cut(bearing_void(body_z).translate((0, bearing_y_offset)))
    .faces(">Z")
    .workplane()
    .pushPoints([(-linear_rod_spacing/2, 0), (linear_rod_spacing/2, 0)])
    .hole(linear_rod_d)
)

for x in [bearing_d/2+m5_r+5,
          linear_rod_spacing/2-linear_rod_d-1.5,
          linear_rod_spacing/2+linear_rod_d+1.5]:
    for r in [1, -1]:
        body = body.cut(m5_hole(body_y).translate((x*r, 0, 0)))

replay(body)

Replay is not enabled. To do so call 'enable_replay()'. Falling back to 'show()'
Done, using side car 'Z'


In [63]:
(a, b) = body.faces(">Y").workplane(offset = -body_y/2).split(True, True).all()

show(a.translate((0, 10)) + b.translate((0, -10)))

Done, using side car 'Z'


In [51]:
exporters.export(a, '/tmp/a.stl')
exporters.export(b, '/tmp/b.stl')